In [26]:
from GroundTruth import groundTruth
import shapely
from shapely.geometry import Polygon


def IoU_calculator(bbox,bbox_aux):
    area_of_intersection = int((bbox.intersection(bbox_aux)).area)
    area_of_union=int((bbox.union(bbox_aux)).area)
    IoU=area_of_intersection/area_of_union
    return IoU



def IoUs_perImage(detectedPeopleDict, pathOfAnnots):
    groundTruthDictionary=groundTruth(pathOfAnnots)
    IoUDictionay=dict()
    
    totalPeople=0
    for key, value in detectedPeopleDict.items():
        peopleDetected=len(value)
        bboxesPerImage=groundTruthDictionary[key]
        peoplePerImage=int(len(bboxesPerImage)/4)
        
        IoU=[]
        for i in range(peopleDetected):
            # Cogemos los ground truth por orden para analizar
            bboxDetected=shapely.geometry.box(value[i][0],value[i][1],value[i][2],value[i][3])
            
            IoUValueAux=0
            for x in range(peoplePerImage):
                
                auxX=x*4
                bboxPrior=shapely.geometry.box(bboxesPerImage[auxX],bboxesPerImage[auxX+1],bboxesPerImage[auxX+2],bboxesPerImage[auxX+3])
                
                # Si intersecta lo analizamos para calcular el IoU
                if bboxDetected.intersects(bboxPrior):
                    IoUValue=round(IoU_calculator(bboxDetected,bboxPrior),2)
                
                if IoUValue>IoUValueAux:
                    IoUValueAux=IoUValue
                    
        
            if IoUValueAux>0.5:
                IoU.append(IoUValueAux)
                
        totalPeople=totalPeople+peoplePerImage
        auxDict=[]
        auxDict.append([peoplePerImage])
        auxDict.append(IoU)
        IoUDictionay[key]=auxDict

    IoUDictionay["totalPeople"]=totalPeople
    
    return IoUDictionay

In [33]:
# Ejemplo
pathOfAnnots=r'''C:\Users\arqc074\Desktop\pruebas\annots'''
diccionario={'test_00000.jpg': [[8, 3, 85, 95], [121, 28, 160, 119], [85, 43, 146, 108]],
 'test_00001.jpg': [[17, 26, 349, 528],
  [328, 215, 581, 557],
  [464, 143, 636, 637]],
 'test_00002.jpg': [[8, 17, 146, 121]]}
pathOfAnnots=r'''C:\Users\arqc074\Desktop\keras-yolo3-master\dataset\annots'''

diccionario=IoUs_perImage(diccionario, pathOfAnnots)

print(diccionario)


{'test_00000.jpg': [[4], [0.77, 0.77, 0.65]], 'test_00001.jpg': [[4], [0.69, 0.69]], 'totalPeople': 9, 'test_00002.jpg': [[1], [0.86]]}
